## XGBoost

In [ ]:
!pip install --upgrade scikit-learn
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error

# Charger les données
file_path = "df_merged_bretagne_filtre.csv"
df = pd.read_csv(file_path, sep=None, engine="python")

# Nettoyer les noms des colonnes
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("%", "pct")
    .str.replace("é", "e")
)

# Convertir les valeurs numériques
for col in df.select_dtypes(include=["object"]).columns:
    if col not in ["nom_prenom", "parti_politique", "couleur_politique"]:  # Éviter de convertir le texte
        df[col] = df[col].str.replace(",", ".").str.replace(" ", "")
        df[col] = pd.to_numeric(df[col], errors="coerce")  # 'coerce' convertit les erreurs en NaN

# Encoder les variables catégoriques avec One-Hot Encoding
df = pd.get_dummies(df, columns=["parti_politique", "couleur_politique"], drop_first=True)

# Sélectionner les variables prédictives et la variable cible
X = df.drop(columns=["pct_voix_obtenu", "nom_prenom"])  # Supprimer le texte inutile
y = df["pct_voix_obtenu"]

# Normaliser les données numériques
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Entraîner le modèle XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Faire des prédictions
y_pred_xgb = xgb_model.predict(X_test)

# Évaluer le modèle
r2_xgb = r2_score(y_test, y_pred_xgb)
# Calculer la RMSE manuellement
mse_xgb = mean_squared_error(y_test, y_pred_xgb)  # Calculer la MSE
rmse_xgb = np.sqrt(mse_xgb)  # Calculer la RMSE à partir de la MSE

print(f"XGBoost - R²: {r2_xgb:.4f}")
print(f"XGBoost - RMSE: {rmse_xgb:.4f}")


XGBoost - R²: 0.8619
XGBoost - RMSE: 6.4236
